TODO:
 - For each track:
     - Compute distance from track start to closest electron position
     - Compute angle between track and closest electron
     
     ...
     - Compute distance from starting point to electron entry
     - Compute angle wrt to electron incidence angle
         - (Combine in IP vector)
     - Plot standardized features -> consider transforms

In [ ]:
#!git remote add upstream https://github.com/yandexdataschool/mlhep2017.git
#!git remote -v
#!git fetch upstream master
#!git merge upstream/master -m "Merge"
#!git push origin

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
!du -h /data_mlhep2017/episode1/*

7.5K	/data_mlhep2017/episode1/DS_1_electron_test.csv
9.0K	/data_mlhep2017/episode1/DS_1_electron_train.csv
94M	/data_mlhep2017/episode1/DS_1_test.csv
103M	/data_mlhep2017/episode1/DS_1_train.csv


In [3]:
# Read track dataset
dtracks = pd.read_csv('/data_mlhep2017/episode1/DS_1_train.csv', index_col=0)

# Read shower dataset
dshower = pd.read_csv('/data_mlhep2017/episode1/DS_1_electron_train.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
#print ("Tracks: (shape =", dtracks.shape)
#dtracks.head()


In [ ]:
#print ("Electron showers:")
#dshower.head()

In [4]:
# Read test datasets
dtracks_test = pd.read_csv('/data_mlhep2017/episode1/DS_1_test.csv', index_col=0)
dshower_test = pd.read_csv('/data_mlhep2017/episode1/DS_1_electron_test.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
#print ("Test track set:")
#dtracks_test.head()

In [ ]:
#print ("Test electron set:")
#dshower_test.head()

## Limit train set
**TODO:** remove

In [ ]:
dtracks_signal = dtracks[dtracks.signal == True][:50000]
#print (dtracks_signal.iloc[0])
dtracks_bgnd = dtracks[dtracks.signal == False][:50000]
#print (dtracks_bgnd.iloc[0])
dtracks = pd.concat([dtracks_signal, dtracks_bgnd])
#dtracks.shape
dtracks = dtracks.sample(frac=1)
dtracks.head()
#print (dtracks.iloc[0])

In [ ]:
dtracks.tail()

## Feature engineering

#### Distance to electron position, angle between track and electron

In [ ]:
def dist(x1, y1, z1, x2, y2, z2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

def index_of_closest_electron(edataset, x, y, z):
    dmin = np.min(dist(x, y, z, edataset['X'], edataset['Y'], edataset['Z']))
    return np.where(dist(x, y, z, edataset['X'], edataset['Y'], edataset['Z']) == dmin)[0][0]

# Compute distance between track start and closest electron, append this to the dataframe
if not 'dist_to_electron' in list(dtracks):
    dtracks['dist_to_electron'] = pd.Series(np.zeros(dtracks.shape[0]), index=dtracks.index)
if not 'angle_to_electron' in list(dtracks):
    dtracks['angle_to_electron'] = pd.Series(np.zeros(dtracks.shape[0]), index=dtracks.index)

for index, row in dtracks.iterrows():
    
    try:
        closest_e = dshower.iloc[index_of_closest_electron(dshower, row['X'], row['Y'], row['Z'])]
        #print (closest_e)
    
        dtracks.set_value(index, 'dist_to_electron', dist(closest_e['X'], closest_e['Y'], closest_e['Z'], row['X'], row['Y'], row['Z']))
        dtracks.set_value(index, 'angle_to_electron', dist(closest_e['TX'], closest_e['TY'], 0, row['TX'], row['TY'], 0))

    except:
        dtracks.set_value(index, 'dist_to_electron', 99999)
        dtracks.set_value(index, 'angle_to_electron', 99999)
        
    if index % 10000 == 0:
        print ("index:", index)
        #print (row)

        
# Do the same for the test set
if not 'dist_to_electron' in list(dtracks_test):
    dtracks_test['dist_to_electron'] = pd.Series(np.zeros(dtracks_test.shape[0]), index=dtracks_test.index)
if not 'angle_to_electron' in list(dtracks_test):
    dtracks_test['angle_to_electron'] = pd.Series(np.zeros(dtracks_test.shape[0]), index=dtracks_test.index)

for index, row in dtracks_test.iterrows():
    
    try:
        closest_e = dshower_test.iloc[index_of_closest_electron(dshower_test, row['X'], row['Y'], row['Z'])]
        dtracks_test.set_value(index, 'dist_to_electron', dist(closest_e['X'], closest_e['Y'], closest_e['Z'], row['X'], row['Y'], row['Z']))
        dtracks_test.set_value(index, 'angle_to_electron', dist(closest_e['TX'], closest_e['TY'], 0, row['TX'], row['TY'], 0))
    except:
        dtracks_test.set_value(index, 'dist_to_electron', 99999)
        dtracks_test.set_value(index, 'angle_to_electron', 99999)
    
# Save the output
dtracks.to_csv('DS_1_train_augmented.csv', header=True)
dtracks_test.to_csv('DS_1_test_augmented.csv', header=True)


index: 0
index: 10000
index: 20000
index: 30000
index: 40000
index: 50000
index: 60000
index: 70000
index: 80000
index: 90000
index: 100000
index: 110000
index: 120000
index: 130000
index: 140000
index: 150000
index: 160000
index: 170000
index: 180000
index: 190000
index: 200000
index: 210000
index: 220000
index: 230000
index: 240000
index: 250000
index: 260000
index: 270000
index: 280000
index: 290000
index: 300000
index: 310000
index: 320000
index: 330000
index: 340000
index: 350000
index: 360000
index: 370000
index: 380000
index: 390000
index: 400000
index: 410000
index: 420000
index: 430000
index: 440000
index: 450000
index: 460000
index: 470000
index: 480000
index: 490000
index: 500000
index: 510000
index: 520000
index: 530000
index: 540000
index: 550000
index: 560000
index: 570000
index: 580000
index: 590000
index: 600000
index: 610000
index: 620000
index: 630000
index: 640000
index: 650000
index: 660000
index: 670000
index: 680000
index: 690000
index: 700000
index: 710000
index:

## Plot before scaling

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

for feature in list(dtracks):
    print (feature)
    plt.title(feature)
    #plt.hist( dtracks[feature])
    plt.hist(dtracks[dtracks['signal'] == 0][feature], histtype='step', label='bgnd', normed=True)
    plt.hist(dtracks[dtracks['signal'] == 1][feature], histtype='step', label='signal', normed=True)
    plt.legend(loc='best')
    plt.show()

## Feature scaling

In [ ]:
# First, strip the targets from features

feature_list = ['X', 'Y', 'Z', 'TX', 'TY', 'chi2', 'dist_to_electron', 'angle_to_electron']
#feature_list = ['X', 'Y', 'Z', 'TX', 'TY', 'chi2']

X = np.array(dtracks[feature_list].values[:])
y = np.array(dtracks['signal'].values[:])
print ("X.shape:", X.shape)
print ("Y.shape:", y.shape)
print (X[:3])
print (y[:3])

# Validation split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

## Create model and train

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

#model = SVC(kernel='rbf', probability=True)
#model = DecisionTreeClassifier()
model = MLPClassifier(hidden_layer_sizes=(40, ))

model.fit(X_train, y_train)
train_predictions = model.predict_proba(X_train)[:, 1]

# Validate
val_predictions = model.predict_proba(X_val)[:, 1]

print('y_val:', y_val[:3])
print('val_pred:', val_predictions[:3])

# ROC
from sklearn.metrics import roc_curve, auc, roc_auc_score
fpr_train, tpr_train, _ = roc_curve(y_train, train_predictions)
roc_auc = auc(fpr_train, tpr_train)
plt.plot(fpr_train, tpr_train, label='train')
print ("AUC train:", roc_auc_score(y_train, train_predictions))

fpr_val, tpr_val, _ = roc_curve(y_val, val_predictions)
roc_auc = auc(fpr_val, tpr_val)
plt.plot(fpr_val, tpr_val, label='test')
plt.legend(loc='best')
plt.show()

print ("AUC val:", roc_auc_score(y_val, val_predictions))

## Test set predictions

In [ ]:
X_test = np.array(dtracks_test[feature_list].values[:])
X_test = scaler.transform(X_test)

test_predictions = model.predict_proba(X_test)[:, 1]

output = pd.DataFrame(test_predictions, columns=['Prediction'])
output.index.name = 'Id'
output.to_csv('predictions.csv', header=True)

In [6]:
!top

=top - 07:43:43 up 181 days, 22:47,  0 users,  load average: 1.00, 1.18, 1.15
Tasks:  47 total,   2 running,   9 sleeping,   0 stopped,  36 zombie
%Cpu(s):  6.4 us,  0.3 sy,  0.0 ni, 93.2 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 50316992 total, 10362384 free,  6854744 used, 33099864 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 43031120 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
 2067 root      20   0 2484456 466372  24060 R 106.7  0.9  33:12.59 python      
    1 root      20   0  378596  76544   3772 S   0.0  0.2   4:12.42 jupyterhub+ 
   60 root      20   0 2308108 310112    448 S   0.0  0.6   8:13.33 python      
   92 root      20   0 3452968 265664   1264 S   0.0  0.5   0:24.34 python      
  266 root      20   0       0      0      0 Z   0.0  0.0   0:00.00 sh          
  288 root      20   0       0      0      0 Z   0.0  0.0   0:00.00 git         
  295 root      20   0       0      0      0 Z   0.0